# Benchmarking Aura features

In [1]:
import os
import timeit

import numpy as np
import seaborn as sns
from tqdm import tqdm
from dotenv import load_dotenv

from graphdatascience.aura_sessions import AuraSessions
from graphdatascience.query_runner.aura_db_arrow_query_runner import AuraDbConnectionInfo

In [3]:
load_dotenv("credentials.env")

CLIENT_ID = os.environ.get("CLIENT_ID")
CLIENT_SECRET = os.environ.get("CLIENT_SECRET")

DB_PASSWORD = os.environ.get("DB_PASSWORD")
DB_ID = os.environ.get("DB_ID")

db_connection_info = AuraDbConnectionInfo(
    f"neo4j+s://{DB_ID}-{os.environ['AURA_ENV']}.databases.neo4j-dev.io", ("neo4j", DB_PASSWORD)
)

In [5]:
sessions = AuraSessions(db_connection_info, (CLIENT_ID, CLIENT_SECRET))
session_name = "benchmark-session"
session_pw = "my-password"

print("Starting GDS session")
if session_name in [i.name for i in sessions.list_sessions()]:
    gds = sessions.connect(session_name, session_pw)
else:
    gds = sessions.create_gds(session_name, session_pw, "24GB")

Starting GDS session


In [22]:
def measure(func, setup, iterations, warmup_iterations):
    pbar = tqdm(total=iterations + warmup_iterations)

    def wrapper():
        value = func()
        pbar.update(1)
        return value

    pbar.set_description("Warmup")
    warmup = timeit.repeat(wrapper, setup=setup, number=1, repeat=warmup_iterations)

    pbar.set_description("Measurement")
    measurement = timeit.repeat(wrapper, setup=setup, number=1, repeat=iterations)

    pbar.close()

    return {"iterations": measurement, "mean": np.mean(measurement), "avg": np.average(measurement)}

## Projection

In [ ]:
def run_remote_projection(query, concurrency):
    with gds.graph.project("graph", query, concurrency=concurrency) as G:
        ()

In [ ]:
run_remote_projection(
    """
    CYPHER runtime = parallel
    MATCH (u)
    OPTIONAL MATCH (u)-[r]->(t)
    RETURN gds.graph.project.remote(u, t, {})
    """,
    concurrency=1,
)

### Project entire graph, structure only

#### No parallel Runtime

In [ ]:
data = {}
for concurrency in [8]:
    result = measure(
        lambda: run_remote_projection(
            """
            MATCH (u)
            OPTIONAL MATCH (u)-[r]->(t)
            RETURN gds.graph.project.remote(u, t, null)
            """,
            concurrency=concurrency,
        ),
        lambda: gds.graph.drop("graph", failIfMissing=False),
        iterations=1,
        warmup_iterations=1,
    )
    data[concurrency] = result

In [ ]:
values = {k: v["mean"] for k, v in list(data.items())}

plot = sns.barplot(values)
plot.set(xlabel="concurrency", ylabel="average runtime")

#### Parallel Runtime

In [ ]:
data = {}
for concurrency in [8]:
    result = measure(
        lambda: run_remote_projection(
            """
            CYPHER runtime = parallel
            MATCH (u)
            OPTIONAL MATCH (u)-[r]->(t)
            RETURN gds.graph.project.remote(u, t, null)
            """,
            concurrency=concurrency,
        ),
        lambda: (),
        iterations=1,
        warmup_iterations=1,
    )
    data[concurrency] = result

plot = sns.barplot({k: v["mean"] for k, v in data.items()})
plot.set(xlabel="concurrency", ylabel="average runtime")

### Project entire graph, with properties and labels

In [ ]:
query = """ CYPHER runtime = parallel
            MATCH (u)
            OPTIONAL MATCH (u)-[r]->(t)
            RETURN gds.graph.project.remote(u, t, {
                sourceNodeLabels: labels(u),
                sourceNodeProperties: {id: id(u)},
                targetNodeLabels: labels(t),
                targetNodeProperties: {id: id(t)},
                relationshipType: type(r),
                relationshipProperties: {id: id(r)}
            })
        """

data = {}
for concurrency in [8]:
    result = measure(
        lambda: run_remote_projection(query, concurrency=concurrency), lambda: (), iterations=10, warmup_iterations=5
    )
    data[concurrency] = result

plot = sns.barplot({k: v["mean"] for k, v in data.items()})
plot.set(xlabel="concurrency", ylabel="average runtime")

## Write back

In [8]:
G, _ = gds.graph.project(
    "graph",
    """
    CYPHER runtime = parallel
    MATCH (u)
    OPTIONAL MATCH (u)-[r]->(t)
    RETURN gds.graph.project.remote(u, t, {})
    """,
    concurrency=4,
)

In [9]:
G = gds.graph.get("graph")

### Node properties

In [10]:
gds.degree.mutate(G, mutateProperty="degree")

nodePropertiesWritten                                              29987836
mutateMillis                                                              0
centralityDistribution    {'min': 0.0, 'max': 9514.062492370605, 'p90': ...
postProcessingMillis                                                      0
preProcessingMillis                                                       0
computeMillis                                                            12
configuration             {'orientation': 'NATURAL', 'mutateProperty': '...
Name: 0, dtype: object

In [24]:
result = measure(
    lambda: gds.graph.nodeProperties.write(G, node_properties=["degree"]),
    lambda: gds.run_cypher("MATCH (n) CALL { WITH n REMOVE n.degree} IN TRANSACTIONS  OF 2000 ROWS"),
    iterations=1,
    warmup_iterations=1,
)

print(f"scalar property write-back: {result}")


Warmup:   0%|          | 0/2 [00:00<?, ?it/s]

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.nodeProperties.write`: Caused by: com.neo4j.gds.shaded.org.apache.arrow.flight.FlightRuntimeException: INTERNAL: org.apache.arrow.flight.FlightRuntimeException: UNKNOWN: java.util.concurrent.ExecutionException: java.lang.RuntimeException: java.lang.NullPointerException: Cannot invoke "org.neo4j.internal.kernel.api.security.LoginContext.authorize(org.neo4j.internal.kernel.api.security.LoginContext$IdLookup, String, org.neo4j.internal.kernel.api.security.AbstractSecurityLog)" because "loginContext" is null
java.util.concurrent.ExecutionException: java.lang.RuntimeException: java.lang.NullPointerException: Cannot invoke "org.neo4j.internal.kernel.api.security.LoginContext.authorize(org.neo4j.internal.kernel.api.security.LoginContext$IdLookup, String, org.neo4j.internal.kernel.api.security.AbstractSecurityLog)" because "loginContext" is null}

In [ ]:
gds.fastRP.mutate(G, mutateProperty="embedding", embeddingDimension=10, iterationWeights=[1.0])

In [ ]:
result = measure(
    lambda: gds.graph.nodeProperties.write(G, node_properties=["embedding"]),
    lambda: gds.run_cypher("MATCH (n) CALL { WITH n REMOVE n.embedding} IN TRANSACTIONS  OF 2000 ROWS"),
    iterations=1,
    warmup_iterations=1,
)

print(f"scalar property write-back: {result}")

### Relationship

In [ ]:
matched_nodes = gds.run_cypher("MATCH (n)-[r]->() WITH count(r) as degree, n WHERE degree > 5 RETURN id(n) ORDER BY id(n) DESC LIMIT 2").squeeze()

In [ ]:
gds.allShortestPaths.delta.mutate(
    G, sourceNode=matched_nodes[0], mutateRelationshipType="MUTATED_RELS", concurrency=6
)

In [ ]:
gds.graph.relationship.write(G, relationship_type="MUTATED_RELS", relationship_property="totalCost")

In [ ]:
# cleanup rels

writted_rels_count = gds.run_cypher("MATCH ()-[r:MUTATED_RELS]->() RETURN count(r)")
print(f"written rels: {writted_rels_count}")
gds.run_cypher("MATCH ()-[r:MUTATED_RELS]->() CALL { WITH r DELETE r} IN TRANSACTIONS  OF 2000 ROWS")

In [ ]:
result = timeit.repeat(
    lambda: gds.run_cypher("MATCH (n)-[r:KNN_RELS]->() DELETE r"),
    lambda: gds.graph.relationship.write(G, relationship_type="KNN_RELS", relationship_property="score"),
    number=1,
    repeat=iterations,
)

print(f"relationships: {result}")

## Cleanup

In [ ]:
gds.graph.drop("graph")

In [ ]:
sessions.delete_gds(session_name)

In [ ]:
# cleanup

from graphdatascience.aura_api import AuraApi

aura_api = AuraApi(CLIENT_ID, CLIENT_SECRET)
aura_api.delete_instance(DB_ID)